### Import libraries

In [1]:

from llama_index.llms.openai_like import OpenAILike

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings
from llama_index.core import SimpleDirectoryReader
from llama_index.core import (
   
    Settings,
    StorageContext,
    VectorStoreIndex,
)

from llama_index.core.llms import ChatMessage
from llama_index.vector_stores.milvus import MilvusVectorStore

from llama_index.embeddings.huggingface import HuggingFaceEmbedding

/home/user/.cache/pypoetry/virtualenvs/speak-diar-vSM-dlU2-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Parameter & models & Settings

In [3]:
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

similarity_top_k = 3
llm = OpenAILike(
        model="meta-llama/Llama-3.1-70B-Instruct",
        api_base="<YOUR_API_URI>",
        api_key="test123",
        
        temperature=0,
        max_tokens=1024
    )


### Loading Documents

In [2]:
documents = SimpleDirectoryReader(
    "<PATH_TO_FOLDER_WHERE_ALL_DOCUMENTS_ARE_PRESENT>",
).load_data()


### Contextual Node Creation

In [34]:
def create_contextual_nodes(nodes_):
    """Function to create contextual nodes for a list of nodes"""
    nodes_modified = []
    for node in nodes_:
        new_node = copy.deepcopy(node)
        messages = [
            ChatMessage(role="system", content="You are helpful AI Assitant."),
            ChatMessage(
                role="user",
                content=[
                    
                    {
                        "text": prompt_chunk.format(CHUNK_CONTENT=node.text),
                        "type": "text",
                    },
                ],
            ),
        ]
        new_node.metadata["context"] = str(
            llm.chat(
                messages,
            )
        )
        nodes_modified.append(new_node)

    return nodes_modified



### Contextual Embdedding

In [35]:

def create_embedding_retriever_contextual(nodes_, similarity_top_k=2):
    """Function to create an embedding retriever for a list of nodes"""
    vector_store = MilvusVectorStore(
            uri="http://localhost:19530", collection_name="scenarios_contextual", dim=384,overwrite=True
        )
   
    storage_context = StorageContext.from_defaults(
            vector_store=vector_store
        )
    index = VectorStoreIndex(
            nodes_, storage_context=storage_context
        )
   
    return index





### Normal Embedding For comaparison....

In [ ]:
def create_embedding_retriever(nodes_, similarity_top_k=2):
    """Function to create an embedding retriever for a list of nodes"""
    vector_store = MilvusVectorStore(
            uri="http://localhost:19530", collection_name="scenarios", dim=384,overwrite=True
        )
   
    storage_context = StorageContext.from_defaults(
            vector_store=vector_store
        )
    index = VectorStoreIndex(
            nodes_, storage_context=storage_context
        )
   
    return index


In [36]:
def set_node_ids(nodes_):
    """Function to set node ids for a list of nodes"""

    for index, node in enumerate(nodes_):
        node.id_ = f"node_{index}"

    return nodes_


In [37]:
from llama_index.core.node_parser import SentenceSplitter

node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=200)

nodes = node_parser.get_nodes_from_documents(documents, show_progress=False)

# set node ids

nodes = set_node_ids(nodes)
nodes_contextual = create_contextual_nodes(nodes)

In [41]:
embedding_retriever = create_embedding_retriever(
    nodes, similarity_top_k=3
)


contextual_embedding_retriever = create_embedding_retriever_contextual(
    nodes_contextual, similarity_top_k=3    
)

### Normal Index retriever

In [ ]:
index = embedding_retriever.as_query_engine(llm=llm,similarity_top_k=3)
index.query("<Your_QUESTION>")

### Contextual embedding retriever

In [49]:
ce_index = contextual_embedding_retriever.as_query_engine(llm=llm,similarity_top_k=3)
ce_index.query("<Your_QUESTION>")

Response(response=' The updated guidance in the April 2024 NFIP Flood Insurance Manual requires the insurer to send a BRCV notice to the agent to obtain the updated BRCV and maintain supporting documentation in the policy file for Residential Condominium Buildings.', source_nodes=[NodeWithScore(node=TextNode(id_='node_656', embedding=None, metadata={'file_path': '/home/user/Aryan/Customer%20Support%20Agent%20using%20Gaudi%203/rec_call/benchmark.mp3.txt', 'file_name': 'benchmark.mp3.txt', 'file_type': 'text/plain', 'file_size': 1018779, 'creation_date': '2024-10-22', 'last_modified_date': '2024-10-22', 'context': 'assistant:  The provided chunk is part of the NFIP Flood Insurance Manual, specifically in Appendix I: Policyholder Communications, section IV, which deals with Severe Repetitive Loss Property Notices.'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'f